In [1]:
import dlib
import cv2
import numpy as np
import os

# Путь к моделям
file_path = r"C:\Users\marus\Desktop\face_det\shape_predictor_68_face_landmarks.dat"
file2_path = r"C:\Users\marus\Desktop\face_det\dlib_face_recognition_resnet_model_v1.dat"

# Инициализация детектора лиц dlib и предиктора
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(file_path)
facerec = dlib.face_recognition_model_v1(file2_path)

def process_images(directory):
    face_descriptors = {}
    for root, dirs, files in os.walk(directory):
        person_name = os.path.basename(root)
        if not files:
            continue
        
        face_descriptors[person_name] = []
        for file in files:
            if file.endswith('.jpg'):  # Adjust for other image formats if needed
                image_path = os.path.join(root, file)
                img = cv2.imread(image_path)
                img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                faces = detector(img_rgb, 1)

                for face in faces:
                    shape = sp(img_rgb, face)
                    face_descriptor = facerec.compute_face_descriptor(img_rgb, shape)
                    face_descriptors[person_name].append(np.array(face_descriptor))
    return face_descriptors

# Пример использования:
path_images = r"C:\Users\marus\Desktop\face_det\images"
all_face_descriptors = process_images(path_images)

# Вывод дескрипторов лиц для каждого человека
for person, descriptors in all_face_descriptors.items():
    print(f"Person: {person}, Number of descriptors: {len(descriptors)}")

# Пример: вычисление среднего дескриптора для каждого человека
average_face_descriptors = {person: np.mean(descriptors, axis=0) for person, descriptors in all_face_descriptors.items()}
for person, avg_descriptor in average_face_descriptors.items():
    print(f"Average descriptor for {person}: {avg_descriptor}")


Person: Albert Einstein, Number of descriptors: 1
Person: Elon Musk, Number of descriptors: 1
Person: Jack Ma, Number of descriptors: 1
Person: Jimmy, Number of descriptors: 1
Person: Johny Deep, Number of descriptors: 1
Person: Sensei, Number of descriptors: 1
Person: Tom Cruise, Number of descriptors: 1
Average descriptor for Albert Einstein: [-0.10211133  0.07293976 -0.00381446  0.00239105 -0.01901988  0.00290474
 -0.00175958 -0.07520515  0.18389821 -0.09677823  0.09998492  0.02215774
 -0.21410671 -0.06802894  0.0137916   0.10393197 -0.07113071 -0.103681
 -0.18206173 -0.13303807 -0.10222684  0.10192621  0.03503226 -0.08703459
 -0.03859901 -0.38931301 -0.03473296 -0.1289461   0.06377003 -0.12797004
  0.03244732  0.07396106 -0.15450938 -0.02686325  0.03273351  0.08275661
 -0.0679832   0.00232644  0.21968247  0.09359004 -0.19853656  0.01435234
  0.02810051  0.27216014  0.21042195 -0.01963543  0.0350909  -0.08131593
  0.1029959  -0.18332645  0.10065594  0.09308953  0.15631975  0.0814094

In [2]:
import numpy as np
import json

# Предполагаем, что reference_face_descriptor уже вычислен
reference_face_descriptor = np.array(avg_descriptor)

# Конвертация в список и запись в файл
path = r"C:\Users\marus\Desktop\face_det\model.txt"
with open(path, 'w') as file:
    json.dump(reference_face_descriptor.tolist(), file)
    print('Data written successfully.')

Data written successfully.


In [3]:
# Чтение дескриптора из файла и конвертация обратно в numpy array
with open(path, 'r') as file:
    reference_face_descriptor = np.array(json.load(file))
    print('Data read successfully.')

print(reference_face_descriptor)


Data read successfully.
[-1.28195584e-02  2.11016685e-01  5.28323762e-02 -3.69853750e-02
 -1.46934405e-01  1.15457788e-01 -3.61238047e-02 -5.43848984e-03
  2.11270034e-01 -7.55740851e-02  2.55477309e-01  5.62203228e-02
 -2.09902912e-01 -4.68763933e-02 -1.13581218e-01  8.57401341e-02
 -1.60647318e-01 -8.05149972e-02 -1.08819194e-01 -1.25245795e-01
  4.02797908e-02  8.00316036e-02 -4.44252603e-02  2.42271051e-02
 -2.72808254e-01 -2.18593284e-01  1.10115083e-02 -1.09413922e-01
  6.83635473e-02 -2.42917404e-01 -3.96085158e-03  1.14397727e-01
 -1.50533617e-01 -2.18289346e-03 -2.75326613e-02  4.68454733e-02
 -9.43566635e-02 -3.63735966e-02  1.79342851e-01  1.60234906e-02
 -9.65825766e-02  1.42852664e-01 -3.94274592e-02  3.21485728e-01
  2.64273703e-01  8.08668975e-03  1.83332711e-04 -9.34071913e-02
  2.60004550e-01 -3.10941815e-01  6.39181957e-02  1.91965550e-01
  7.95793831e-02  8.52261186e-02  1.31202355e-01 -1.99288487e-01
 -2.07107235e-02  9.62831378e-02 -1.42318115e-01  1.87878683e-02
 

In [4]:
import face_recognition
import os
import pickle

# Путь к каталогу с известными лицами
known_faces_dir = r"C:\Users\marus\Desktop\face_det\images"  # Замените на ваш путь

known_face_names = []
known_face_encodings = []

print("Loading known faces...")
for person_dir in os.listdir(known_faces_dir):
    person_path = os.path.join(known_faces_dir, person_dir)
    if os.path.isdir(person_path):
        for filename in os.listdir(person_path):
            if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust for other image formats
                image_path = os.path.join(person_path, filename)
                known_face_image = face_recognition.load_image_file(image_path)
                known_face_encodings_list = face_recognition.face_encodings(known_face_image)
                if known_face_encodings_list:  # Ensure encoding is not empty
                    known_face_encoding = known_face_encodings_list[0]
                    known_face_names.append(person_dir)
                    known_face_encodings.append(known_face_encoding)
                    print(f"Loaded encoding for {person_dir}")

print(f"Total known faces loaded: {len(known_face_names)}")

# Сохраняем энкодинги в файл known_faces.pkl
with open('known_faces.pkl', 'wb') as f:
    pickle.dump((known_face_names, known_face_encodings), f)

print("Encodings saved to known_faces.pkl")


Loading known faces...
Loaded encoding for Albert Einstein
Loaded encoding for Elon Musk
Loaded encoding for Jack Ma
Loaded encoding for Jimmy
Loaded encoding for Johny Deep
Loaded encoding for Sensei
Loaded encoding for Tom Cruise
Total known faces loaded: 7
Encodings saved to known_faces.pkl


In [5]:
import dlib
import face_recognition
import cv2
import datetime
import os
import pickle
import openpyxl

# Load model paths (replace with your actual paths)
file_path = r"C:\Users\marus\Desktop\face_det\shape_predictor_68_face_landmarks.dat"
file2_path = r"C:\Users\marus\Desktop\face_det\dlib_face_recognition_resnet_model_v1.dat"

# Initialize detectors
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(file_path)
facerec = dlib.face_recognition_model_v1(file2_path)

# Load known faces from pickle file
with open('known_faces.pkl', 'rb') as f:
    known_face_names, known_face_encodings = pickle.load(f)

print(f"Total known faces loaded: {len(known_face_names)}")

# Path to the log file
log_file_path = 'log.xlsx'

# Create a new Excel file and write the header if it doesn't exist
if not os.path.exists(log_file_path):
    wb = openpyxl.Workbook()
    ws = wb.active
    ws.append(['Name', 'Time', 'Action'])
    wb.save(log_file_path)

def log_entry(name, action):
    current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    wb = openpyxl.load_workbook(log_file_path)
    ws = wb.active
    ws.append([name, current_time, action])
    wb.save(log_file_path)
    print(f"Logged {action} for {name} at {current_time}")

def get_last_action(name):
    wb = openpyxl.load_workbook(log_file_path)
    ws = wb.active
    for row in reversed(list(ws.iter_rows(values_only=True))):
        if row[0] == name:
            return row[2]
    return None

print("Initialization complete.")

# Dictionary to store the last detected time of each person
last_detected = {}

# Function to detect faces in a video
def detect_faces_in_video(video_path):
    cap = cv2.VideoCapture(video_path)

    # Define the desired frame width and height (720p)
    frame_width = 720
    frame_height = 720

    # Time to wait before confirming detection (in seconds)
    wait_time = 2

    # Time threshold for giving up on detection (in seconds)
    time_threshold = 10

    start_time = datetime.datetime.now()

    detected_name = None
    detected_time = None

    while True:
        ret, frame = cap.read()
        if not ret:
            print("No more frames to read.")
            break

        # Resize the frame to 720p while maintaining the aspect ratio
        height, width, _ = frame.shape
        scaling_factor = min(frame_width / width, frame_height / height)
        new_width = int(width * scaling_factor)
        new_height = int(height * scaling_factor)
        resized_frame = cv2.resize(frame, (new_width, new_height))

        # Convert frame to grayscale for face detection
        gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
        faces = detector(gray_frame, 1)

        # Check if time threshold has been reached
        current_time = datetime.datetime.now()
        if (current_time - start_time).total_seconds() > time_threshold:
            print("No face detected within the time threshold.")
            break

        if not faces:
            print("No faces detected in the frame.")
            continue  # Skip if no faces are detected

        print(f"Detected {len(faces)} faces in the frame.")

        # Convert dlib rectangles to the format that face_recognition expects
        face_locations = [(face.top(), face.right(), face.bottom(), face.left()) for face in faces]

        # Loop through detected faces
        for face_location, face_encoding in zip(face_locations, face_recognition.face_encodings(resized_frame, face_locations)):
            matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
            print(f"Face encoding matches: {matches}")
            name = "Unknown"

            # If a match is found, get the name from the known faces list
            if True in matches:
                first_match_index = matches.index(True)
                name = known_face_names[first_match_index]
                print(f"Match found: {name}")

                today_str = current_time.strftime('%Y-%m-%d')

                # Check if the person was detected recently
                if name in last_detected:
                    last_entry_date = last_detected[name].strftime('%Y-%m-%d')
                    if last_entry_date == today_str:
                        print(f"{name} has already entered today")
                        continue  # Skip if the person has already entered today

                if detected_name == name:
                    # Check if the person has been detected for the wait time
                    if (current_time - detected_time).total_seconds() >= wait_time:
                        last_action = get_last_action(name)
                        if last_action == "entry":
                            log_entry(name, "exit")
                        else:
                            log_entry(name, "entry")
                        last_detected[name] = current_time  # Update the last detected time after logging
                        detected_name = None  # Reset the detected name
                        detected_time = None  # Reset the detected time
                else:
                    detected_name = name
                    detected_time = current_time
                    print(f"Detection started for {name} at {detected_time}")

    cap.release()
    cv2.destroyAllWindows()
    print("Finished processing all frames.")

# Example usage
video_path = r"C:\Users\marus\Desktop\face_det\videos\Sensei.mp4"  # Replace with your video path
detect_faces_in_video(video_path)


Total known faces loaded: 7
Initialization complete.
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, False, False, True, False]
Match found: Sensei
Detection started for Sensei at 2024-11-28 14:41:05.216848
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, False, False, True, False]
Match found: Sensei
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, False, False, True, False]
Match found: Sensei
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, False, False, True, False]
Match found: Sensei
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, False, False, True, False]
Match found: Sensei
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, False, False, True, False]
Match found: Sensei
Logged exit for Sensei at 2024-11-28 14:41:08
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, False, False, True, False]
M

In [4]:
# import dlib
# import face_recognition
# import os
# import csv
# import datetime

# # Load model paths (replace with your actual paths)
# file_path = r"C:\Users\marus\Desktop\face_det\shape_predictor_68_face_landmarks.dat"
# file2_path = r"C:\Users\marus\Desktop\face_det\dlib_face_recognition_resnet_model_v1.dat"

# # Load reference face descriptors (modify this section)
# known_faces_dir = r"C:\Users\marus\Desktop\face_det\images"  # Directory containing known faces
# known_face_names = []
# known_face_encodings = []

# print("Loading known faces...")
# for person_dir in os.listdir(known_faces_dir):
#     person_path = os.path.join(known_faces_dir, person_dir)
#     if os.path.isdir(person_path):
#         for filename in os.listdir(person_path):
#             if filename.endswith(".jpg") or filename.endswith(".png"):  # Adjust for other image formats
#                 image_path = os.path.join(person_path, filename)
#                 known_face_image = face_recognition.load_image_file(image_path)
#                 known_face_encodings_list = face_recognition.face_encodings(known_face_image)
#                 if known_face_encodings_list:  # Ensure encoding is not empty
#                     known_face_encoding = known_face_encodings_list[0]
#                     # Use directory name as the person's name
#                     known_face_names.append(person_dir)
#                     known_face_encodings.append(known_face_encoding)
#                     print(f"Loaded encoding for {person_dir}")

# print(f"Total known faces loaded: {len(known_face_names)}")

# # Initialize detectors
# detector = dlib.get_frontal_face_detector()
# sp = dlib.shape_predictor(file_path)
# facerec = dlib.face_recognition_model_v1(file2_path)

# # Path to the log file
# log_file_path = 'log.csv'

# # Create a new CSV file and write the header if it doesn't exist
# if not os.path.exists(log_file_path):
#     with open(log_file_path, 'w', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow(['Name', 'Time', 'Action'])

# def log_entry(name, action):
#     current_time = datetime.datetime.now().strftime('%Y-%m-%d %H:%M:%S')
#     with open(log_file_path, 'a', newline='') as file:
#         writer = csv.writer(file)
#         writer.writerow([name, current_time, action])
#     print(f"Logged {action} for {name} at {current_time}")



# print("Initialization complete.")


Loading known faces...
Loaded encoding for Albert Einstein
Loaded encoding for Elon Musk
Loaded encoding for Jack Ma
Loaded encoding for Jimmy
Loaded encoding for Johny Deep
Loaded encoding for Sensei
Loaded encoding for Tom Cruise
Total known faces loaded: 7
Initialization complete.


In [17]:
# import cv2
# import datetime

# # Function to determine the last action of the person
# def get_last_action(name):
#     if not os.path.exists(log_file_path):
#         return None  # No log file exists, so no actions recorded
#     with open(log_file_path, 'r', newline='') as file:
#         reader = csv.reader(file)
#         rows = list(reader)
#         for row in reversed(rows):
#             if row[0] == name:
#                 return row[2]  # Return the last action
#     return None  # No action found for this person

# # Dictionary to store the last detected time of each person
# last_detected = {}

# video_path = r"C:\Users\marus\Desktop\face_det\videos\jimmy.mp4"  # Replace with your video path
# cap = cv2.VideoCapture(video_path)

# # Define the desired frame width and height (720p)
# frame_width = 720
# frame_height = 720

# # Time to wait before confirming detection (in seconds)
# wait_time = 2

# # Time threshold for giving up on detection (in seconds)
# time_threshold = 10

# start_time = datetime.datetime.now()

# detected_name = None
# detected_time = None

# while True:
#     ret, frame = cap.read()
#     if not ret:
#         print("No more frames to read.")
#         break

#     # Resize the frame to 720p while maintaining the aspect ratio
#     height, width, _ = frame.shape
#     scaling_factor = min(frame_width / width, frame_height / height)
#     new_width = int(width * scaling_factor)
#     new_height = int(height * scaling_factor)
#     resized_frame = cv2.resize(frame, (new_width, new_height))

#     # Convert frame to grayscale for face detection
#     gray_frame = cv2.cvtColor(resized_frame, cv2.COLOR_BGR2GRAY)
#     faces = detector(gray_frame, 1)

#     # Check if time threshold has been reached
#     current_time = datetime.datetime.now()
#     if (current_time - start_time).total_seconds() > time_threshold:
#         print("No face detected within the time threshold.")
#         break

#     if not faces:
#         print("No faces detected in the frame.")
#         continue  # Skip if no faces are detected

#     print(f"Detected {len(faces)} faces in the frame.")

#     # Convert dlib rectangles to the format that face_recognition expects
#     face_locations = [(face.top(), face.right(), face.bottom(), face.left()) for face in faces]

#     # Loop through detected faces
#     for face_location, face_encoding in zip(face_locations, face_recognition.face_encodings(resized_frame, face_locations)):
#         matches = face_recognition.compare_faces(known_face_encodings, face_encoding)
#         print(f"Face encoding matches: {matches}")
#         name = "Unknown"

#         # If a match is found, get the name from the known faces list
#         if True in matches:
#             first_match_index = matches.index(True)
#             name = known_face_names[first_match_index]
#             print(f"Match found: {name}")

#             today_str = current_time.strftime('%Y-%m-%d')

#             # Check if the person was detected recently
#             if name in last_detected:
#                 last_entry_date = last_detected[name].strftime('%Y-%m-%d')
#                 if last_entry_date == today_str:
#                     print(f"{name} has already entered today")
#                     continue  # Skip if the person has already entered today

#             if detected_name == name:
#                 # Check if the person has been detected for the wait time
#                 if (current_time - detected_time).total_seconds() >= wait_time:
#                     last_action = get_last_action(name)
#                     if last_action == "entry":
#                         log_entry(name, "exit")
#                     else:
#                         log_entry(name, "entry")
#                     last_detected[name] = current_time  # Update the last detected time after logging
#                     detected_name = None  # Reset the detected name
#                     detected_time = None  # Reset the detected time
#             else:
#                 detected_name = name
#                 detected_time = current_time
#                 print(f"Detection started for {name} at {detected_time}")

# cap.release()
# cv2.destroyAllWindows()
# print("Finished processing all frames.")


Detected 1 faces in the frame.
Face encoding matches: [False, False, False, True, False, False, False]
Match found: Jimmy
Detection started for Jimmy at 2024-08-05 19:53:55.293466
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, True, False, False, False]
Match found: Jimmy
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, True, False, False, False]
Match found: Jimmy
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, True, False, False, False]
Match found: Jimmy
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, True, False, False, False]
Match found: Jimmy
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, True, False, False, False]
Match found: Jimmy
Logged entry for Jimmy at 2024-08-05 19:53:58
Detected 1 faces in the frame.
Face encoding matches: [False, False, False, True, False, False, False]
Match found: Jimmy
Jimmy has already entered today
Detected 1